In [1]:
# !git clone https://github.com/TheMITTech/shakespeare

In [2]:
# from glob import glob
# files = glob("./shakespeare/**/*.html")

In [3]:
# import shutil
# import os
# 
# # os.mkdir('./data')
# destination_folder = './data/'
# 
# for html_file in files[1:]:
#     html_file = html_file.replace("\\", "/")
#     shutil.move(html_file, destination_folder + html_file.split("/")[-1])

In [34]:
import dotenv

from langchain.document_loaders import BSHTMLLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

dotenv.load_dotenv()

True

In [30]:
bloomz_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b7")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(bloomz_tokenizer, chunk_size=500, chunk_overlap=50, separator="\n")
embeddings = HuggingFaceEmbeddings()
persist_directory = "vector_db"


In [32]:
rebuild_db = False

if rebuild_db:
    bshtml_dir_loader = DirectoryLoader('./data/', loader_cls=BSHTMLLoader)
    data = bshtml_dir_loader.load()
    documents = text_splitter.split_documents(data)
    vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)
    vectordb.persist()
    vectordb = None

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
doc_retriever = vectordb.as_retriever()

In [35]:
# llm = HuggingFacePipeline.from_model_id(
#     model_id="bigscience/bloomz-1b7", 
#     task="text-generation", 
#     model_kwargs={"temperature" : 0, "max_length" : 500},
#     device=0,
# )

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4"
)

### Final Chain


In [36]:
shakespeare_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever)

In [37]:
for i in range(20):
    q = questions[i]["Q"]
    r = shakespeare_qa.run(q)
    questions[i]["R"] = r

    print(q)
    print(r)
    print("\n")


What is the significance of the handkerchief in 'Othello'?
The handkerchief in 'Othello' is a significant symbol. It was first given to Desdemona by Othello, representing his love and trust for her. However, when it is lost and ends up in the hands of Iago, it becomes a tool of manipulation. Iago uses it to convince Othello that Desdemona has been unfaithful, leading to tragic consequences. Thus, the handkerchief symbolizes love, trust, betrayal, and manipulation in the play.


How does 'Macbeth' explore the theme of ambition and its consequences?
The text provided does not give specific information on how 'Macbeth' explores the theme of ambition and its consequences.


In 'King Lear,' how does Shakespeare portray the relationship between madness and wisdom?
The text provided does not give specific information on how Shakespeare portrays the relationship between madness and wisdom in 'King Lear.'


How does the concept of fate play out in 'Romeo and Juliet'?
The context provided does n

In [38]:
import json
with open("results/openai_gpt4.json", "w+") as f:
    f.write(json.dumps(questions))